In [2]:
import pandas as pd
import numpy as np

# Load the new dataset for inspection before training
new_file_path = r"C:\Users\snpdp\pm2.5-Forecast-Dashboard\Data_csv\merge_data\jsps-m-1d.csv"
new_df = pd.read_csv(new_file_path)

# Display basic information about the dataset
new_df.info(), new_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7138 entries, 0 to 7137
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   timestamp    7138 non-null   object 
 1   humidity     6531 non-null   float64
 2   pm_2_5       7138 non-null   float64
 3   pm_2_5_sp    7074 non-null   float64
 4   temperature  6598 non-null   float64
dtypes: float64(4), object(1)
memory usage: 279.0+ KB


(None,
     timestamp   humidity     pm_2_5  pm_2_5_sp  temperature
 0  2021-08-15  82.633751  29.242424        NaN    28.392102
 1  2021-08-16  87.845353  19.432905        NaN    26.644278
 2  2021-08-17  90.980718  13.293243        NaN    26.299335
 3  2021-08-18  77.261290  15.163043        NaN    28.296894
 4  2021-08-19  78.807031   7.994888        NaN    27.969023)

In [11]:
import pandas as pd

# ฟังก์ชันตรวจจับ outliers โดยใช้ IQR method
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (data < lower_bound) | (data > upper_bound)

# โหลดข้อมูล
file_path = r"C:\Users\snpdp\pm2.5-Forecast-Dashboard\Data_csv\merge_data\jsps-m-1d.csv"
df = pd.read_csv(file_path)

# คัดลอก DataFrame สำหรับการทำความสะอาด
cleaned_df = df.copy()

# ตรวจสอบและลบ Outliers
for column in ['pm_2_5', 'humidity', 'temperature', 'pm_2_5_sp']:
    if column in cleaned_df.columns:
        outliers = detect_outliers_iqr(cleaned_df[column])
        cleaned_df = cleaned_df[~outliers]

# เติมค่าที่หายไป
for col in ['humidity', 'temperature']:
    if col in cleaned_df.columns:
        cleaned_df[col] = cleaned_df[col].fillna(cleaned_df[col].median())

if 'pm_2_5_sp' in cleaned_df.columns:
    cleaned_df['pm_2_5_sp'] = cleaned_df['pm_2_5_sp'].interpolate(method='linear')

if 'pm_2_5' in cleaned_df.columns:
    cleaned_df['pm_2_5'] = cleaned_df['pm_2_5'].fillna(cleaned_df['pm_2_5'].mean())

# เติมค่าหายไปจากข้อมูลก่อนหน้า (Forward Fill) และข้อมูลถัดไป (Backward Fill)
cleaned_df.fillna(method='ffill', inplace=True)
cleaned_df.fillna(method='bfill', inplace=True)

# ลบข้อมูลที่ซ้ำกัน
cleaned_df.drop_duplicates(inplace=True)

# จัดการ timestamp ซ้ำกัน และลบเวลาออก
if 'timestamp' in cleaned_df.columns:
    cleaned_df['timestamp'] = cleaned_df['timestamp'].astype(str).str.strip()
    cleaned_df['timestamp'] = pd.to_datetime(cleaned_df['timestamp'], errors='coerce')

    # แปลง timestamp ให้เก็บเฉพาะ "วัน" ไม่มีเวลา
    cleaned_df['date'] = cleaned_df['timestamp'].dt.date

    # รวมข้อมูลที่มีวันเดียวกัน (ใช้ค่าเฉลี่ย)
    cleaned_df = cleaned_df.groupby('date').mean().reset_index()

    # แปลง 'date' กลับเป็น datetime โดยไม่มีเวลา
    cleaned_df['timestamp'] = pd.to_datetime(cleaned_df['date'])

    # ลบคอลัมน์ 'date' ที่ใช้ช่วยคำนวณ
    cleaned_df.drop(columns=['date'], inplace=True)

# บันทึกไฟล์ที่ทำความสะอาดแล้ว
cleaned_file_path = r"C:\Users\snpdp\pm2.5-Forecast-Dashboard\Data_csv\merge_data\cleaned_jsps-m-1d.csv"
cleaned_df.to_csv(cleaned_file_path, index=False)

# แสดงผลข้อมูลที่ทำความสะอาดแล้ว
print("✅ Data Cleaning Completed. Cleaned file saved at:", cleaned_file_path)
print(cleaned_df.head())


✅ Data Cleaning Completed. Cleaned file saved at: C:\Users\snpdp\pm2.5-Forecast-Dashboard\Data_csv\merge_data\cleaned_jsps-m-1d.csv
   timestamp   humidity     pm_2_5  pm_2_5_sp  temperature
0 2021-08-15  82.633751  29.242424   1.100546    28.392102
1 2021-08-16  87.845353  19.432905   1.100546    26.644278
2 2021-08-18  77.261290  15.163043   1.100546    28.296894
3 2021-08-19  78.807031   7.994888   1.100546    27.969023
4 2021-08-20  81.790799   9.832737   1.100546    27.503250


C:\Users\snpdp\AppData\Local\Temp\ipykernel_1868\101219567.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cleaned_df.fillna(method='ffill', inplace=True)
C:\Users\snpdp\AppData\Local\Temp\ipykernel_1868\101219567.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cleaned_df.fillna(method='bfill', inplace=True)
